In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_seasonal.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_seasonal.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_seasonal.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_seasonal.nc']

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_prcptot_seasonal.nc']

# FOR RCP45 

In [11]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds45 = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds45.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [13]:
i = 0
df45list = []
for ds in listds45:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    dfp452 =  df453.reset_index()
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45list.append(Region1i45)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [14]:
df45list[0]

,index,lat,lon,time,prcptot_p10,prcptot_p50,prcptot_p90
0,1,66.623306,-89.045212,1950-12-01,25.03,34.60,47.51
1,2,66.623306,-89.045212,1951-12-01,22.80,30.75,46.68
2,3,66.623306,-89.045212,1952-12-01,16.67,29.58,50.60
3,4,66.623306,-89.045212,1953-12-01,19.05,25.30,34.58
4,5,66.623306,-89.045212,1954-12-01,24.16,33.77,44.65
...,...,...,...,...,...,...,...
11430895,222182194,40.041039,-74.045807,2095-12-01,257.28,332.76,387.51
11430896,222182195,40.041039,-74.045807,2096-12-01,203.98,256.04,382.54
11430897,222182196,40.041039,-74.045807,2097-12-01,223.94,291.67,427.14
11430898,222182197,40.041039,-74.045807,2098-12-01,254.92,289.26,361.23


# FOR RCP85 

In [15]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 152)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1949-12-01 1950-12-01 ... 2100-12-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 3, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

In [13]:
i = 0
df45list = []
for ds in listds45:
    print ("Dataset number: ", i)
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    dfp452 =  df453.reset_index()
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45list.append(Region1i45)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Transformed in Celsiuds and rounded
DF added to the list


In [14]:
df45list[0]

,index,lat,lon,time,prcptot_p10,prcptot_p50,prcptot_p90
0,1,66.623306,-89.045212,1950-12-01,25.03,34.60,47.51
1,2,66.623306,-89.045212,1951-12-01,22.80,30.75,46.68
2,3,66.623306,-89.045212,1952-12-01,16.67,29.58,50.60
3,4,66.623306,-89.045212,1953-12-01,19.05,25.30,34.58
4,5,66.623306,-89.045212,1954-12-01,24.16,33.77,44.65
...,...,...,...,...,...,...,...
11430895,222182194,40.041039,-74.045807,2095-12-01,257.28,332.76,387.51
11430896,222182195,40.041039,-74.045807,2096-12-01,203.98,256.04,382.54
11430897,222182196,40.041039,-74.045807,2097-12-01,223.94,291.67,427.14
11430898,222182197,40.041039,-74.045807,2098-12-01,254.92,289.26,361.23


## Get the percentiles

In [ ]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [ ]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [ ]:
#dfmean85

In [ ]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [ ]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# OBSERVED DATA

In [ ]:
dsOBs= xr.open_mfdataset(filesobs)

In [ ]:
dsOBs

## Convert in Dataframe

In [ ]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [ ]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})

In [ ]:
dfObs

In [ ]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [ ]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [ ]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [ ]:
df2m

# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [ ]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [ ]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

In [ ]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [ ]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [ ]:
dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 


In [ ]:
dfmeanObs

In [ ]:
listdf2m = []

for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs2 = dfmeanObs.reset_index()
    print("Merging 3 cases")
    df3m = pd.merge(dfmean45UA2, dfmeanObs2, how='left', on= ["time", name] )
    print ("4/8... Split by subregion")
    df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                               var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs']]
    if (var == "tg_mean" or var== "tn_mean" or var==):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    if (var == "solidprcptot"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        print ("                   "+name2, name)
        dfsrc = dfsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            if (var == "tg_mean" or var== "tn_mean" or var==):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            if (var == "solidprcptot"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_annual.csv")
    
    
                            
       